# MNIST data classification: Keras and input pipeline

This notebook shows how to use input pipelines with Keras.

In [1]:
import numpy as np
import math
import tensorflow as tf

In [2]:
tf.version.VERSION

'2.3.0'

In [4]:
def decode(serialized_example):
    """Parses an image and label from the given `serialized_example`."""
    features = tf.io.parse_single_example(
        serialized_example,
        features={
            'image_raw': tf.io.FixedLenFeature([], tf.string),
            'label': tf.io.FixedLenFeature([], tf.int64),
        })
    label = tf.cast(features['label'], tf.int32)
    image = tf.io.decode_raw(features['image_raw'], tf.uint8)
    return image, label


def normalize(image, label):
    """Convert `image` from [0, 255] -> [-0.5, 0.5] floats."""
    image = tf.cast(image, tf.float32) * (1. / 255) - 0.5
    return image, label

In [5]:
dataset = tf.data.TFRecordDataset('../tfrecords/train.tfrecords')
dataset = dataset.map(decode)
dataset = dataset.map(normalize)
dataset = dataset.shuffle(6000)
dataset = dataset.batch(256)
dataset = dataset.repeat(5)

In [6]:
test_set = tf.data.TFRecordDataset('../tfrecords/test.tfrecords')
test_set = test_set.map(decode)
test_set = test_set.map(normalize)
test_set = test_set.batch(256)

In [7]:
model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(784,)),
    tf.keras.layers.Dense(128, activation=tf.nn.relu),
    tf.keras.layers.Dense(10, activation=tf.nn.softmax)
])

In [8]:
opt = tf.keras.optimizers.Adam(0.001)

In [9]:
model.compile(optimizer=opt,
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [10]:
fit = model.fit(dataset,
                epochs=5,
                validation_data=test_set)

Epoch 1/5
1175/1175 [==============================] - 16s 13ms/step - loss: 0.2689 - accuracy: 0.9227 - val_loss: 0.1386 - val_accuracy: 0.9591
Epoch 2/5
1175/1175 [==============================] - 15s 12ms/step - loss: 0.1039 - accuracy: 0.9700 - val_loss: 0.0941 - val_accuracy: 0.9717
Epoch 3/5
1175/1175 [==============================] - 14s 12ms/step - loss: 0.0651 - accuracy: 0.9808 - val_loss: 0.0830 - val_accuracy: 0.9737
Epoch 4/5
1175/1175 [==============================] - 15s 13ms/step - loss: 0.0448 - accuracy: 0.9869 - val_loss: 0.0764 - val_accuracy: 0.9760
Epoch 5/5
1175/1175 [==============================] - 14s 12ms/step - loss: 0.0317 - accuracy: 0.9908 - val_loss: 0.0758 - val_accuracy: 0.9759
